In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import keras

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from nilearn.input_data import NiftiMasker
from nilearn import datasets, plotting, image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import confusion_matrix
from nilearn.plotting import plot_anat, show, plot_stat_map, plot_matrix

Using TensorFlow backend.


In [2]:
# Designate input file
haxby_ds = datasets.fetch_haxby(subjects=[4], fetch_stimuli=True)

# 'func' is a list of filenames: one for each subject
func_file = haxby_ds.func[0]

# Standardizing
mask_vt_file = haxby_ds.mask_vt[0]
masker = NiftiMasker(mask_img=mask_vt_file, standardize=True)

# Load the behavioral data that I will predict
beh_label = pd.read_csv(haxby_ds.session_target[0], sep=" ")

# # Regressout the resting state
nonrest_task_mask = (beh_label['labels'] != 'rest')

#select data
X = masker.fit_transform(func_file)[nonrest_task_mask]
y = beh_label['labels'][nonrest_task_mask]

In [3]:
y.unique()

array(['face', 'chair', 'scissors', 'shoe', 'scrambledpix', 'house',
       'cat', 'bottle'], dtype=object)

In [4]:
y = y.replace('face', 1).replace('chair', 2).replace('scissors', 3).replace('shoe', 4).replace('scrambledpix', 5).replace('house', 6).replace('cat', 7).replace('bottle', 8)

In [5]:
y.unique()

array([1, 2, 3, 4, 5, 6, 7, 8])

In [6]:
#shuffle and split the data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle=True, random_state = 0)

#standarize features caling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## Initializing the ANN

In [7]:
classifier = Sequential()

In [8]:
# Adding the input layer and the first hidden layer
classifier.add(Dense(output_dim = 338, init = 'uniform', activation = 'relu', input_dim = 675))

/home/srastegarnia/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=675, units=338, kernel_initializer="uniform")`
  


In [9]:
# Adding the second hidden layer
classifier.add(Dense(output_dim = 338, init = 'uniform', activation = 'relu'))

/home/srastegarnia/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=338, kernel_initializer="uniform")`
  


In [10]:
# Adding the output layer
classifier.add(Dense(output_dim = 1, init = 'uniform', activation = 'sigmoid'))

/home/srastegarnia/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=1, kernel_initializer="uniform")`
  


In [11]:
# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [12]:
X_train.shape

(777, 675)

In [13]:
# Fitting the ANN to the Training set
classifier.fit(X_train, y_train, epochs = 10)

Epoch 1/10
777/777 [==============================] - 1s 1ms/step - loss: -159.5563 - accuracy: 0.1184
Epoch 2/10
777/777 [==============================] - 0s 262us/step - loss: -2782.0778 - accuracy: 0.1197
Epoch 3/10
777/777 [==============================] - 0s 236us/step - loss: -14830.7619 - accuracy: 0.1197
Epoch 4/10
777/777 [==============================] - 0s 245us/step - loss: -50142.8000 - accuracy: 0.1197
Epoch 5/10
777/777 [==============================] - 0s 241us/step - loss: -126717.2258 - accuracy: 0.1197
Epoch 6/10
777/777 [==============================] - 0s 259us/step - loss: -267764.0477 - accuracy: 0.1197
Epoch 7/10
777/777 [==============================] - 0s 245us/step - loss: -487256.2708 - accuracy: 0.1197
Epoch 8/10
777/777 [==============================] - 0s 236us/step - loss: -811797.0204 - accuracy: 0.1197
Epoch 9/10
777/777 [==============================] - 0s 251us/step - loss: -1260065.6300 - accuracy: 0.1197
Epoch 10/10
777/777 [===============

In [14]:
# Making the predictions and evaluating the model
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

In [15]:
print(y_pred)

[[ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]]


In [16]:
# cm = confusion_matrix(y_test, y_pred)
# print(cm)